In [6]:
import numpy as np
import pandas as pd
import wfdb
import matplotlib.pyplot as plt
import ast
import os
import tensorflow as tf
#from keras.utils import to_categorical

#import tensorflow_datasets as tfds
import datetime

from sklearn.utils import resample
import pywt

In [7]:
path='/Users/xianghuihuang/Desktop/WFDB/'
tf_save_path = '/Users/xianghuihuang/Desktop/tfrecord_4labels'

In [8]:
def get_label_info(path):
  def add_path(str2):
      return path + "HR"+str2.split("_")[0]
  def aggregate_diagnostic(y_dic):
      tmp = []
      for key in y_dic.keys():
          if key in agg_df.index:
              tmp.append(agg_df.loc[key].diagnostic_class)
      if len(tmp) == 0:
        return "NL"
      return list(set(tmp))[0]
  Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
  Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

      # Load scp_statements.csv for diagnostic aggregation
  agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
  agg_df = agg_df[agg_df.diagnostic == 1]

      # Apply diagnostic superclass
      #target label of the classification
  Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
  Y['new_filename'] = Y.filename_hr.map(add_path)
  Y['strat_fold'] = Y.strat_fold
  return Y[["new_filename","diagnostic_superclass","age","sex","strat_fold"]]

In [9]:
y = get_label_info(path)
y.head()

,new_filename,diagnostic_superclass,age,sex,strat_fold
ecg_id,,,,,
21837,/Users/xianghuihuang/Desktop/WFDB/HR21837,NORM,68.0,0,9
21836,/Users/xianghuihuang/Desktop/WFDB/HR21836,NORM,64.0,1,8
21835,/Users/xianghuihuang/Desktop/WFDB/HR21835,STTC,59.0,1,2
21834,/Users/xianghuihuang/Desktop/WFDB/HR21834,NORM,93.0,0,4
21833,/Users/xianghuihuang/Desktop/WFDB/HR21833,STTC,67.0,1,7


In [10]:
y.diagnostic_superclass.value_counts()

NORM    9526
MI      5485
STTC    2878
HYP     1830
CD      1709
NL       407
Name: diagnostic_superclass, dtype: int64

In [17]:
y = y[y.diagnostic_superclass !="NL"]
y.diagnostic_superclass.value_counts()

NORM    9488
STTC    3957
CD      3011
MI      2538
HYP     2434
Name: diagnostic_superclass, dtype: int64

In [18]:
y = y[y.diagnostic_superclass !="NORM"]
y.diagnostic_superclass.value_counts()

STTC    3957
CD      3011
MI      2538
HYP     2434
Name: diagnostic_superclass, dtype: int64

In [19]:
y = y.dropna(how='any',axis=0) 
y.diagnostic_superclass.value_counts()

STTC    3928
CD      2985
MI      2530
HYP     2418
Name: diagnostic_superclass, dtype: int64

In [20]:
#numeric_label = {"diagnostic_superclass":{"NORM":0,"CD":1,"MI":2,"STTC":3,"HYP":4}}
numeric_label = {"diagnostic_superclass":{"MI":0,"STTC":1,"CD":2,"HYP":3}}
y = y.replace(numeric_label)

In [22]:
y.diagnostic_superclass.value_counts()

1    3928
2    2985
0    2530
3    2418
Name: diagnostic_superclass, dtype: int64

In [23]:
# Split data into train and test
test_fold = 10
val_fold = 9
train_fold = [1,2,3,4,5,6,7,8]

# Train
train_df = y[(y.strat_fold.isin(train_fold))]
# Test
test_df = y[(y.strat_fold == test_fold)]
#test_df = df_new.loc[df_new["strat_fold"]. isin(test_fold)]
val_df = y[(y.strat_fold == val_fold) ]


# #split the train test dataset
# train_df, test_df = train_test_split(df_new, test_size = 0.2)
# #split the train validation dataset
# train_df, val_df = train_test_split(train_df, test_size = 0.2)

In [24]:
train_df.diagnostic_superclass.value_counts()

1    3127
2    2371
0    2043
3    1935
Name: diagnostic_superclass, dtype: int64

In [29]:
3127/len(train_df)

0.32999155761924864

In [30]:
2371/len(train_df)

0.2502110595187843

In [31]:
2043/len(train_df)

0.21559729843815956

In [32]:
1935/len(train_df)

0.20420008442380752

In [33]:
test_df.diagnostic_superclass.value_counts()

1    386
2    318
0    254
3    232
Name: diagnostic_superclass, dtype: int64

In [34]:
386/len(test_df)

0.3243697478991597

In [35]:
318/len(test_df)

0.2672268907563025

In [36]:
254/len(test_df)

0.2134453781512605

In [37]:
232/len(test_df)

0.1949579831932773

In [38]:
val_df.diagnostic_superclass.value_counts()

1    415
2    296
3    251
0    233
Name: diagnostic_superclass, dtype: int64

In [39]:
#shuffle the dataframe and re_index the dataset
def shuffle_dataset(df):
  df = df.sample(frac=1)
  df = df.reset_index()
  return df

In [40]:
# def resample_df_two_class(df_majority,df_minority,n_samples ):
#     #up sampling the miniority class of dataset
#   df_minority_upsampled = resample(df_minority, 
#                                   replace=True,     # sample with replacement
#                                   n_samples= n_samples,    # to match majority class
#                                   random_state=123) # reproducible results
#   df_new = pd.concat([df_majority, df_minority_upsampled], ignore_index= True)
#   #df_new.diagnostic_superclass.value_counts()
#   return df_new

In [41]:
# df_majority = train_df[train_df.diagnostic_superclass==1]
# df_minority = train_df[train_df.diagnostic_superclass==0]
# n_samples = len(df_majority)
# df_new = resample_df_two_class(df_majority,df_minority,n_samples)
# df_new.diagnostic_superclass.value_counts()

In [42]:
# df_majority = train_df[train_df.diagnostic_superclass==0]
# df_minority1 = train_df[train_df.diagnostic_superclass==1]
# df_minority2 = train_df[train_df.diagnostic_superclass==2]
# df_minority3 = train_df[train_df.diagnostic_superclass==3]
# # df_minority4 = train_df[train_df.diagnostic_superclass==4]
# n_samples = len(df_minority2)

# # df_minority1_upsampled = resample(df_minority1, 
# #                                   replace=True,     # sample with replacement
# #                                   n_samples= n_samples,    # to match majority class
# #                                   random_state=123) # reproducible results
# # df_minority2_upsampled = resample(df_minority2, 
# #                                   replace=True,     # sample with replacement
# #                                   n_samples= n_samples,    # to match majority class
# #                                   random_state=123) # reproducible results
# df_minority3_upsampled = resample(df_minority3, 
#                                   replace=True,     # sample with replacement
#                                   n_samples= n_samples,    # to match majority class
#                                   random_state=123) # reproducible results
# # df_minority4_upsampled = resample(df_minority4, 
# #                                   replace=True,     # sample with replacement
# #                                   n_samples= n_samples,    # to match majority class
# #                                   random_state=123) # reproducible results

In [43]:
# df_new = pd.concat([df_majority, df_minority1,df_minority2,df_minority3_upsampled], ignore_index= True)

In [44]:
# df_new.diagnostic_superclass.value_counts()

In [45]:
# train_df = df_new

In [46]:
test_df.diagnostic_superclass.value_counts()

1    386
2    318
0    254
3    232
Name: diagnostic_superclass, dtype: int64

In [47]:
val_df.diagnostic_superclass.value_counts()

1    415
2    296
3    251
0    233
Name: diagnostic_superclass, dtype: int64

In [48]:
print("train dataset information:")
print(train_df.info())
print()
print("test dataset information:")
print(test_df.info())
print()
print("validation dataset information:")
print(val_df.info())

train dataset information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9476 entries, 21835 to 22
Data columns (total 5 columns):
new_filename             9476 non-null object
diagnostic_superclass    9476 non-null int64
age                      9476 non-null float64
sex                      9476 non-null int64
strat_fold               9476 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 444.2+ KB
None

test dataset information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1190 entries, 21826 to 63
Data columns (total 5 columns):
new_filename             1190 non-null object
diagnostic_superclass    1190 non-null int64
age                      1190 non-null float64
sex                      1190 non-null int64
strat_fold               1190 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 55.8+ KB
None

validation dataset information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195 entries, 21788 to 8
Data columns (total 5 columns

In [49]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def save_tfrecords(data, labels, desfile):
    with tf.io.TFRecordWriter(desfile) as writer:
        
       for i in range(len(data)):
           features = tf.train.Features(
                    feature = {
                        "data": _bytes_feature(tf.io.serialize_tensor(data[i])),
                        "label":_bytes_feature(tf.io.serialize_tensor(labels[i])),
    #                     "sex":_int64_feature(sex),
    #                     "age":_float_feature(age)
                    }
                )
           example = tf.train.Example(features = features)
           serialized = example.SerializeToString()
           writer.write(serialized)

In [50]:
def denoise(sig_data,sig_num):
    # wavedec
#     print(record_name)
        #print("denoise data:",sig_data)
        #print(sig_num)
        sig_length = len(sig_data)
        #print("pywt.waverec 1")
        coeffs = pywt.wavedec(data=sig_data, wavelet='db8', level=9)
        #print("pass!")
        cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs
        #print('here')
        # threshold
        threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
        cD1.fill(0)
        cD2.fill(0)
        for i in range(1, len(coeffs) - 2):
           coeffs[i] = pywt.threshold(coeffs[i], threshold)
        #print("pywt.waverec 2")
        rdata = pywt.waverec(coeffs=coeffs, wavelet='db8')
        #print("denoise data:")
        sig_array2 = rdata
        zero_array = np.zeros(sig_length,)
        if max(sig_array2) != 0:
            #rdata = sig_array2[0:sig_length]
            for i in range(len(sig_array2)):
                #print("denoise loop")
                zero_array[i] = sig_array2[i]
            
            rdata = np.nan_to_num(zero_array)
        else:
            print("denoise zero error!")
            return sig_data
        
        #print("denoise return data:",rdata)
        return rdata

In [51]:
def denoise_tfrecord(data,file_path):
    try:
     # Generating numpy array 
      data_array = data.T
      #print(data_array.shape)
      for i, sig_array in enumerate(data_array):
        #print("loop start!")
        sig_array2 = denoise(sig_array,i)
        #print("loop shape:")
        data_array[i] = sig_array2
      data2 = data_array.T
#       if data_array.T.shape[0] != 5000 or data_array.T.shape[1] != 12:
# #       if data_array    
#           print("data_array shape:",data_array.T.shape)
#           print("data shape:",data2.shape)
      return data2        
    except:
      print('Denoise error!')
      print(file_path)
      #print(data.numpy())
      return data

In [45]:
def load_data_all(df,name,label_num):
  path_list = df['new_filename']
  #print(i)
  #age_tensor = tf.constant(df['age'][start_index: end_index],dtype="int8")
  #sex_tensor =tf.constant(df['sex'][start_index: end_index],dtype="int8")
  labels_one_hot = tf.one_hot(df['diagnostic_superclass'],label_num)
  data_list = []
  path_list=path_list.reset_index()
  tf_path = os.path.join(tf_save_path,name+'.tfrecords')
  for j in range(len(path_list)):
    file_path = path_list.new_filename[j]
    
    data = wfdb.rdrecord(file_path).p_signal
    data = denoise_tfrecord(data,file_path).astype("float16")
    #print(data.shape)
    
    if data is None:
        print(file_path,"error!!")
    #save_tfrecords(data, labels_one_hot[j], tf_path)
    data_list.append(data)
    
  data_array = np.array(data_list)
  save_tfrecords(data_array, labels_one_hot, tf_path)
  #dataset = tf.data.Dataset.from_tensor_slices((data_array,age_tensor,sex_tensor,label_one_hot))
  #set tensorflow feature
  #label_tensor = tf.constant(train_df['diagnostic_superclass'][start_index: end_index],2)
  #dataset = tf.data.Dataset.from_tensor_slices((data_array, labels_one_hot))

In [48]:
load_data_all(test_df,"test2",2)